In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score 
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, auc, log_loss
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)



# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
df_original = pd.read_csv('/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df_original.head()

In [ ]:
df_original.dtypes

we can see that Total Charges is object type so now we will convert this to float.

In [ ]:
df_original['TotalCharges'].unique()

In [ ]:
#replacing blank spaces with none
df_original= df_original.replace(' ',np.nan)
#coverting string to float datatype
df_original['TotalCharges'] = df_original['TotalCharges'].astype(float)
df_original['TotalCharges'].dtypes

In [ ]:
df_original.isnull().sum()

In [ ]:
#percentage of missing value
(df_original['TotalCharges'].isnull().sum()/df_original.shape[0])*100

we can see the percentage of missing values in Total Charges is even less than 1 percent. so we can delete it.

In [ ]:
df_original = df_original.dropna()
df_original.isnull().sum()

In [ ]:
#get all categorical columns from dataframe
df_categorical_col  = df_original.select_dtypes(include = 'object').columns
df_categorical_col

In [ ]:
#get all numerical columns from dataframe
df_numerical_col  = df_original.select_dtypes(exclude = 'object').columns
df_numerical_col

In [ ]:
# it will the statistical values of the numerical columns
df_original.describe()

In [ ]:
# checking data is balanced or imbalanced
df_original['Churn'].value_counts()


In [ ]:
slices_churn = df_original['Churn'].value_counts().tolist()
lables = ['No', 'Yes']
colors = ['y', 'g']
plt.pie(slices_churn, labels=lables, colors=colors, startangle=90, autopct='%.1f%%')
plt.show()

In [ ]:
df_original['SeniorCitizen'].value_counts()


In [ ]:

sns.boxplot(data = df_original, y = 'MonthlyCharges', x = 'Churn')

In [ ]:
sns.boxplot(data = df_original, y = 'TotalCharges', x = 'Churn')

It seems quite interesting. In the plot of monthly charges, lower charges users are seems to have lower Churn rates. However, in the ploty of total charges, lower charge users are seems to have hihger Chrun rate.

In [ ]:
plt.figure(figsize=[8,6])
plt.scatter(df_original["TotalCharges"].values, range(df_original.shape[0]))
plt.title("Distribution of Total Charges")
plt.xlabel("Total charges");
plt.ylabel("Number of Occurences")

In [ ]:
df_original.head()

In [ ]:
cols = ['Churn','tenure','MonthlyCharges','TotalCharges' ]
sns.pairplot(df_original[cols],hue = 'Churn')

In [ ]:
df = df_original[df_original.Churn == 'Yes']
sns.distplot(df['TotalCharges'], hist = False, kde = True, label='Yes')

df = df_original[df_original.Churn == 'No']
sns.distplot(df['TotalCharges'], hist = False, kde = True, label='No')
# Plot formatting
plt.legend(prop={'size': 12})
plt.title('Chrun Yes vs No')
plt.xlabel('Total Charges')
plt.ylabel('value')  


In [ ]:
df = df_original[df_original.Churn == 'Yes']
sns.distplot(df['MonthlyCharges'], hist = False, kde = True, label='Yes')

df = df_original[df_original.Churn == 'No']
sns.distplot(df['MonthlyCharges'], hist = False, kde = True, label='No')
# Plot formatting
plt.legend(prop={'size': 12})
plt.title('Chrun Yes vs No')
plt.xlabel('MonthlyCharges')
plt.ylabel('Fequency ')  


In [ ]:
df_original['SeniorCitizen'].value_counts()

In [ ]:
sns.countplot(x='SeniorCitizen', hue="Churn", data=df_original)

In [ ]:
### shorter the contract period, higher probability of churn
ct  = pd.crosstab(df_original.Contract, df_original.Churn)
ct.plot(kind='bar')
plt.title('Churn by Contract')
plt.ylabel("# of chrun")
plt.show()

In [ ]:
pip install plotly

Now we will binarize the categorical columns having two values i.e. yes/no yes = 1, no = 0


In [ ]:
unique_values_columns = []
for i in df_original.columns:
    length = len(df_original[i].unique())
    unique_values_columns.append(length)
print(unique_values_columns)




In [ ]:
#get all the unique values of columns
#df_original.nunique()
col_unique_values = pd.Series(unique_values_columns, 
               index=df_original.columns.tolist(), name='col_unique_values')
col_unique_values

In [ ]:
#get the columns values that has two unique values eg. yes/no, male/female
cols_two_unique_values =col_unique_values[col_unique_values  == 2].index.tolist() 
cols_two_unique_values

In [ ]:

cols_two_unique_values[0:6]

In [ ]:
df_label = df_original.drop('Churn',axis = 1)
df_label.head()


In [ ]:
# instantiate labelencoder object
le = LabelEncoder()
for i in cols_two_unique_values[0:6]:
    df_label[i] =le.fit_transform(df_label[i])
df_label

In [ ]:
df_label.head()

In [ ]:
#Churn is the target variable that is why we manually assign 0 to NO,Yes to 1 value
dict = {'Yes':1,'No':0}
df_original['Churn_label']=df_original['Churn'].map(dict)
df_original.head()

In [ ]:
df_label['Churn_label']=df_original['Churn_label']
df_label.head()

In [ ]:
#customer id seems to be irrelevent column so we are dropping it
df_label=df_label.drop('customerID', axis = 1)


In [ ]:
df =df_label.copy()
df

In [ ]:
#dummification for the remainingg categorical columns
cat_lable_col =col_unique_values[(col_unique_values >2) & col_unique_values[(col_unique_values <5)]] 

df_label = pd.get_dummies(data = df_label,columns = cat_lable_col.index )

df_label.head()


In [ ]:
df_label.head()

In [ ]:
# normalization for columns that has large variance
df_final = df_label[['TotalCharges','tenure','MonthlyCharges']]
df_normalized = (df_final- df_final.mean())/df_final.std()
df_label= df_label.drop(['TotalCharges','tenure','MonthlyCharges'],1)
df_label=pd.concat([df_label,df_normalized], axis = 1)
df_label.head()

**Train test split and Base Model Fitting****

In [ ]:
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
X = df_label.drop(['Churn_label'], axis = 1)
y = df_label['Churn_label']
x_train,x_test,y_train,y_test = train_test_split(X,y)
glm_binom = sm.GLM(y_train,(sm.add_constant(x_train)), family=sm.families.Binomial())
res = glm_binom.fit()
print(res.summary())


**Base Logistic regression model**

In [ ]:
### model fitting using logistic regression
from sklearn.linear_model import LogisticRegression
# initialize logistic regression
lm = LogisticRegression()
fitted=lm.fit(x_train, y_train)
y_pred = lm.predict(x_test)
y_pred_proba = lm.predict_proba(x_test)[:, 1]
scores = lm.score(x_test, y_test)
print("Logistic Regression score is.:" + str(lm.score(x_test,y_test)))


In [ ]:

y_pred_proba = lm.predict_proba(x_test)[:, 1]
[fpr, tpr, thr] = metrics.roc_curve(y_test, y_pred_proba)
print('Train/Test split results:')
print(lm.__class__.__name__+" accuracy is %2.3f" % metrics.accuracy_score(y_test, y_pred))
print(lm.__class__.__name__+" log_loss is %2.3f" % metrics.log_loss(y_test, y_pred_proba))
print(lm.__class__.__name__+" auc is %2.3f" % metrics.auc(fpr, tpr))


**Evalutation Using AUC/ROC**

In [ ]:
fpr, tpr, thres = metrics.roc_curve(y_test, y_pred)
roc_auc = metrics.auc(fpr,tpr)
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

we can see our roc curve is 0.75.

**Model evaluation based on K-fold cross-validation using cross_validate() function**

In [ ]:
# Applying K fold cross validation


scores_accuracy = cross_val_score(lm, X, y, cv=10, scoring='accuracy')
scores_log_loss = cross_val_score(lm, X, y, cv=10, scoring='neg_log_loss')
scores_auc = cross_val_score(lm, X, y, cv=10, scoring='roc_auc')
print('K-fold cross-validation results:')
print(lm.__class__.__name__+" average accuracy is %2.3f" % scores_accuracy.mean())
print(lm.__class__.__name__+" average log_loss is %2.3f" % -scores_log_loss.mean())
print(lm.__class__.__name__+" average auc is %2.3f" % scores_auc.mean())

In [ ]:
plt.figure(figsize=(24,12))
cm_lr = confusion_matrix(y_test, y_pred)
plt.subplot(2,3,2)
plt.title("Logistic Regression Confusion Matrix")
print(sns.heatmap(cm_lr,annot=True,cmap="Blues",fmt="d",cbar=False, annot_kws={"size": 24}))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

**Accuracy is 65%**
**Churn Rate    -->31%**
**NonChurn Rate -->75%**

In [ ]:
#Hyperparameter tuning for Logistic Regression
from sklearn.model_selection import GridSearchCV
penalty = ['l1', 'l2']
C = np.logspace(0, 4, 10)
hyperparameters = {
    'C':C,
    'penalty':penalty
}

h_logmodel = GridSearchCV(lm, hyperparameters, cv=5, verbose=0)
best_logmodel=h_logmodel.fit(X,y)
best_logmodel.best_params_

In [ ]:
### model fitting using logistic regression
from sklearn.linear_model import LogisticRegression
# initialize logistic regression
lm = LogisticRegression(penalty='l2',C=166)
fitted=lm.fit(x_train, y_train)
y_pred = lm.predict(x_test)
y_pred_proba = lm.predict_proba(x_test)[:, 1]
scores = lm.score(x_test, y_test)
print("Logistic Regression score is.:" + str(lm.score(x_test,y_test)))

In [ ]:
C